In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
#reload modules
%load_ext autoreload
%autoreload 2

In [3]:
import join_data as jd

In [4]:
from helpers import *

# Load and join the public data

In [5]:
mergeddfpublic = jd.pipeline(directory = '../data/', FSfilename = 'FSR_221022.csv', FIfilename = 'FI_221022.csv', FWOfilename = 'FWO_221022.csv', FRAfilename = 'FRA_221024.csv')

In [6]:
mergeddfpublic.columns

Index(['OBJECTID', 'SRCategory', 'SRType', 'SRPriority', 'SRSource',
       'SRStatus', 'SRResolution', 'Borough', 'CommunityBoard',
       'ServiceRequestParentGlobalID', 'GlobalID', 'InitiatedDate',
       'SRClosedDate', 'SRCreatedDate', 'SRUpdatedDate', 'Descriptor1',
       'ComplaintType', 'CallerZipCode', 'SRCallerType', 'Latitude',
       'Longitude', 'Census Tract', 'NTA', 'SRGlobalID', 'InspectionType',
       'InspectionStatus', 'InspectionTPCondition', 'InspectionTPStructure',
       'TreePointDBH', 'InsGlobalID', 'InspectionDate', 'InsClosedDate',
       'InsCreatedDate', 'InsUpdatedDate', 'ParentInspectionGlobalID',
       'ReinspectionDate', 'Location', 'WOType', 'WOStatus', 'WOPriority',
       'CancelReason', 'WOCategory', 'WOGlobalID', 'WOClosedDate',
       'CancelDate', 'WOCreatedDate', 'WOUpdatedDate', 'WOEntity',
       'PROJSTARTDATE', 'WOProject', 'RecommendedSpecies', 'Location_WO',
       'RADefect', 'RADefectLocation', 'Failure', 'ImpactTarget',
       'Conse

# Calculate work order delays

First, combine multiple reports of the same incident so that I have one row per unique incident. If there are multiple inspections or work orders, grab date of the first inspection/work order

In [7]:
nodups = mergeddfpublic[['IncidentGlobalID','SRCategory','SRCreatedDate', 'InspectionDate','WOClosedDate','Risk_coded','RiskRating','Borough','InspectionTPCondition', 'InspectionTPStructure',
       'TreePointDBH']].groupby('IncidentGlobalID').agg(
    {'SRCreatedDate': 'min', 'InspectionDate': 'min', 'WOClosedDate': 'min', 'SRCategory': 'first', 'Risk_coded': 'first', 'Borough': 'first', 'RiskRating': 'first', 'InspectionTPCondition': 'first', 'InspectionTPStructure': 'first',
       'TreePointDBH': 'first'}).reset_index()

In [8]:
# calculate inspection and work order delays
nodups['inspection_delay'] = (nodups['InspectionDate'] - nodups['SRCreatedDate']).dt.total_seconds() / 3600 /24

nodups['work_delay'] = (nodups['WOClosedDate'] - nodups['InspectionDate']).dt.total_seconds() / 3600 /24

In [20]:
# nodups[['SRCreatedDate','InspectionDate', 'WOClosedDate']].describe(datetime_is_numeric=True)

# Work order delays for entire dataset

In [10]:
nodups.groupby(['Risk_coded', 'Borough'])['work_delay'].median()

Risk_coded  Borough      
A           Bronx             16.388484
            Brooklyn           7.467645
            Manhattan          4.382326
            Queens             6.377083
            Staten Island      5.178374
B           Bronx             29.111644
            Brooklyn           8.275231
            Manhattan          6.153935
            Queens             5.523611
            Staten Island     12.178148
C           Bronx             90.472789
            Brooklyn          57.314537
            Manhattan         10.674653
            Queens            64.214329
            Staten Island     67.229282
D           Bronx            101.051655
            Brooklyn          45.412083
            Manhattan         25.443287
            Queens            46.221424
            Staten Island     81.078808
E           Bronx            118.201736
            Brooklyn                NaN
            Queens           784.118704
Unknown     Bronx             71.504549
            Br

In [11]:
nodups.groupby(['SRCategory', 'Borough'])['work_delay'].median()

SRCategory           Borough      
Claims               Bronx                   NaN
                     Brooklyn         385.130255
                     Manhattan               NaN
                     Queens                  NaN
                     Staten Island           NaN
Hazard               Bronx              8.629167
                     Brooklyn          16.270197
                     Manhattan          5.289977
                     Queens             7.270787
                     Staten Island      7.930249
Illegal Tree Damage  Bronx            114.000914
                     Brooklyn          88.127818
                     Manhattan         19.319444
                     Queens            64.876053
                     Staten Island     81.797541
Pest/Disease         Bronx            382.103455
                     Brooklyn          82.548964
                     Manhattan        112.770127
                     Queens           180.134097
                     Staten Island

# If an incident got inspected but no work order, pretend it had a really long work order delay

In [12]:
nodups_imputed = nodups.copy()
nodups_imputed.loc[:, 'work_delay'] = nodups_imputed['work_delay'].fillna(10000)

In [13]:
# only look at inspected incidents
nodups_imputed = nodups_imputed.dropna(subset = ['inspection_delay'])

In [14]:
nodups_imputed.groupby(['Risk_coded', 'Borough'])['work_delay'].median()

Risk_coded  Borough      
A           Bronx               21.097685
            Brooklyn             8.365266
            Manhattan            4.395139
            Queens               7.198299
            Staten Island        6.110087
B           Bronx               41.006204
            Brooklyn            10.303044
            Manhattan            6.338542
            Queens               6.527824
            Staten Island       13.875046
C           Bronx              343.638825
            Brooklyn           133.986632
            Manhattan           12.524618
            Queens             185.751047
            Staten Island      141.998519
D           Bronx            10000.000000
            Brooklyn         10000.000000
            Manhattan          127.253889
            Queens           10000.000000
            Staten Island    10000.000000
E           Bronx            10000.000000
            Brooklyn         10000.000000
            Queens           10000.000000
Unknown 

In [15]:
nodups_imputed.groupby(['SRCategory', 'Borough'])['work_delay'].median()

SRCategory           Borough      
Claims               Bronx            10000.000000
                     Brooklyn           385.130255
                     Queens           10000.000000
Hazard               Bronx              246.685694
                     Brooklyn           419.954792
                     Manhattan            7.187106
                     Queens              76.919387
                     Staten Island       28.044745
Illegal Tree Damage  Bronx            10000.000000
                     Brooklyn         10000.000000
                     Manhattan        10000.000000
                     Queens           10000.000000
                     Staten Island    10000.000000
Pest/Disease         Bronx            10000.000000
                     Brooklyn         10000.000000
                     Manhattan        10000.000000
                     Queens           10000.000000
                     Staten Island    10000.000000
Plant Tree           Bronx              476.910

# Only look at 2017 - 2020 like in the paper, and other filtering (e.g., where reports are defined, subset of categories)

In [16]:
# filter to categories that we're analysing in the paper
nodups = nodups[nodups['SRCategory'].isin(['Hazard', 'Remove Tree', 'Root/Sewer/Sidewalk', 'Prune','Illegal Tree Damage'])]

#if these fields are NA we didn't have data to calculate reporting delay
nodups = nodups.dropna(subset = ['RiskRating', 'Borough', 'SRCategory', 'Risk_coded', 'InspectionTPCondition', 'TreePointDBH'])

In [17]:
nodups_rightdate = nodups[(nodups['SRCreatedDate'] >= '2017-06-30') & (nodups['SRCreatedDate'] < '2020-07-01')]

nodups_right_dateimputed = nodups_rightdate.copy()
nodups_right_dateimputed.loc[:, 'work_delay'] = nodups_right_dateimputed['work_delay'].fillna(10000)
# only look at inspected incidents
nodups_right_dateimputed = nodups_right_dateimputed.dropna(subset = ['inspection_delay'])

In [18]:
nodups_rightdate.dropna(subset = ['inspection_delay']).groupby(['Risk_coded', 'Borough'])['work_delay'].median()

Risk_coded  Borough      
A           Bronx             17.202182
            Brooklyn           5.016273
            Manhattan          5.180764
            Queens             6.925162
            Staten Island      8.834097
B           Bronx             38.717882
            Brooklyn           7.333524
            Manhattan          6.309028
            Queens             6.267558
            Staten Island     24.991609
C           Bronx            145.983785
            Brooklyn          69.962025
            Manhattan         11.397917
            Queens            84.372361
            Staten Island    126.019931
D           Bronx            161.926979
            Brooklyn          64.545139
            Manhattan         35.588889
            Queens            76.386285
            Staten Island    130.341302
E           Bronx            118.201736
            Brooklyn                NaN
            Queens           895.154317
Unknown     Bronx            146.188299
            Br

In [19]:
# with imputation
nodups_right_dateimputed.groupby(['Risk_coded', 'Borough'])['work_delay'].median()

Risk_coded  Borough      
A           Bronx               25.121956
            Brooklyn             5.672222
            Manhattan            5.317361
            Queens               7.239653
            Staten Island       11.886979
B           Bronx               57.086493
            Brooklyn             8.670833
            Manhattan            6.372222
            Queens               7.237176
            Staten Island       31.019300
C           Bronx              404.543397
            Brooklyn           131.293727
            Manhattan           14.334375
            Queens             156.236748
            Staten Island      187.110272
D           Bronx            10000.000000
            Brooklyn         10000.000000
            Manhattan          120.514931
            Queens           10000.000000
            Staten Island    10000.000000
E           Bronx            10000.000000
            Brooklyn         10000.000000
            Queens           10000.000000
Unknown 